In [1]:
from FinMind.data import DataLoader 
import pandas as pd



In [47]:
## Get data
stock_no = '006208'
dl = DataLoader()
stock_data = dl.taiwan_stock_daily(
    stock_id=stock_no, 
    start_date='2012-09-30',
    end_date='2024-09-30')
stock_data.to_csv('stock_data.csv', index=False)



2024-09-26 01:53:13.790 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPrice, data_id: 006208


In [48]:
## calculate MA
df = pd.read_csv('stock_data.csv')

def calculate_ma(data, window):
    return data.rolling(window=window).mean()
    
ma_window = 50
df[f'MA_{ma_window}'] = df['close'].rolling(window=50).mean()
df.to_csv('stock_data.csv', index=False)

print(df[[f'MA_{ma_window}', 'close']].tail())




        MA_50   close
2815  105.994  105.35
2816  105.811  106.60
2817  105.626  107.00
2818  105.409  108.05
2819  105.293  109.80


In [49]:
## get average trade volume for 10 days before
df['avg_trade_volume_10d'] = df['Trading_Volume'].rolling(window=10).mean()
print(df['avg_trade_volume_10d'].tail())

2815    7330277.2
2816    7375252.6
2817    7146829.2
2818    6102050.9
2819    6151225.6
Name: avg_trade_volume_10d, dtype: float64


In [50]:
## 買入信號：前一天的關盤價 < 前一天的 MA 且 當天的關盤價 > 當天的 MA
df['is_buy_signal'] = (df['close'].shift(1) < df[f'MA_{ma_window}'].shift(1)) & (df['close'] > df[f'MA_{ma_window}'])

print(df['is_buy_signal'].tail())

2815    False
2816     True
2817    False
2818    False
2819    False
Name: is_buy_signal, dtype: bool


In [58]:
## 賣出信號: 前一天的關盤價 > 前一天的 MA 且 當天的關盤價 < 當天的 MA
df['is_sell_signal'] = (df['close'].shift(1) > df[f'MA_{ma_window}'].shift(1)) & (df['close'] < df[f'MA_{ma_window}'])

print(df['is_sell_signal'])

0       False
1       False
2       False
3       False
4       False
        ...  
2815    False
2816    False
2817    False
2818    False
2819    False
Name: is_sell_signal, Length: 2820, dtype: bool


In [59]:
# 找出所有買入信號的日期
buy_dates = df[df['is_buy_signal']]['date']

# 找出所有賣出信號的日期
sell_dates = df[df['is_sell_signal']]['date']

print("買入信號日期：")
print(buy_dates.tail())

print("\n賣出信號日期：")
print(sell_dates.tail())

買入信號日期：
2713    2024-04-23
2776    2024-07-23
2781    2024-08-01
2792    2024-08-16
2816    2024-09-20
Name: date, dtype: object

賣出信號日期：
2712    2024-04-22
2775    2024-07-22
2777    2024-07-26
2782    2024-08-02
2793    2024-08-19
Name: date, dtype: object


In [56]:
## 找出 均線黃金交叉 與 均線死亡交叉, 以 50 與 200 為例
df['MA_200'] = df['close'].rolling(window=200).mean()
df['MA_50'] = df['close'].rolling(window=50).mean()

df['is_golden_cross'] = (df['MA_50'].shift(1) < df['MA_200'].shift(1)) & (df['MA_50'] > df['MA_200'])
df['is_death_cross'] = (df['MA_50'].shift(1) > df['MA_200'].shift(1)) & (df['MA_50'] < df['MA_200'])

print(f'黃金交叉:\n{df[df['is_golden_cross']]['date']} \n')
print(f'死亡交叉:\n{df[df['is_death_cross']]['date']} \n')



黃金交叉:
388     2014-05-28
485     2014-10-28
622     2015-06-02
627     2015-06-09
632     2015-06-16
636     2015-06-23
653     2015-07-17
914     2016-08-25
1019    2017-02-13
1545    2019-05-20
1820    2020-07-14
2442    2023-03-06
Name: date, dtype: object 

死亡交叉:
321     2014-02-18
396     2014-06-11
618     2015-05-27
624     2015-06-04
631     2015-06-15
635     2015-06-22
652     2015-07-16
761     2015-12-22
942     2016-10-14
1442    2018-11-29
1757    2020-04-08
2238    2022-04-18
Name: date, dtype: object 

